# Scraping Wikipedia to prove or disprove A CURSE

In this notebook, I'm going to have a go at scraping a Wikipedia table, then cross-referencing it with other Wikipedia articles to get a richer dataset, and then doing some summaries to compare to national statistics.

The "Strictly Curse" comes up in discussion every Winter, when celebrity contestants on Strictly Come Dancing are always rumoured to be having affairs with their professional partners & then leaving their spouses. I thought it'd be interesting to see if, statistically, there's anything to it.

## Section 1: Get everything I can from the main table

en.wikipedia.org/wiki/List_of_Strictly_Come_Dancing_contestants has a list of everyone who's competed on Strictly since it started in 2004. I'm going to use that to get the main list of contestants, then have a look into their own wiki pages (later on) to pull out their marital status(es).

In [193]:
# Import the relevant packages

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [194]:
# Get the page, then soupify it.

page_url = "https://en.wikipedia.org/wiki/List_of_Strictly_Come_Dancing_contestants"
page = requests.get(page_url)
soup = BeautifulSoup(page.content,'html.parser')

In [195]:
# Find the right table
main_table = soup.find(class_="wikitable sortable")

# Get the contestants from that table
con_info = main_table.find_all('tr')

# Count the contestants
num = len(list(con_info))

# Set up empty arrays for all the fields
con_names=[]
con_occupations=[]
con_ages =[]
con_partners =[]
con_series=[]
con_dances=[]
con_highscore=[]
con_lowscore=[]
con_avscore=[]
con_placings=[]

# Populate the info fields using the "td" tags
for i in range(1,num):
    con_names.append(con_info[i].find_all("td")[0].get_text().rstrip('\n'))
    con_occupations.append(con_info[i].find_all("td")[1].get_text().rstrip('\n'))
    con_ages.append(con_info[i].find_all("td")[2].get_text().rstrip('\n'))
    con_partners.append(con_info[i].find_all("td")[3].get_text().rstrip('\n'))
    con_series.append(con_info[i].find_all("td")[4].get_text().rstrip('\n'))
    con_dances.append(con_info[i].find_all("td")[6].get_text().rstrip('\n'))
    con_highscore.append(con_info[i].find_all("td")[7].get_text().rstrip('\n'))
    con_lowscore.append(con_info[i].find_all("td")[8].get_text().rstrip('\n'))
    con_avscore.append(con_info[i].find_all("td")[10].get_text().rstrip('\n'))
    con_placings.append(con_info[i].find_all("td")[5].get_text().rstrip('\n'))
    

In [196]:
# Combine all those lists into a table
allinfo = {'Name':con_names,
          'Occupation':con_occupations,
          'Age':con_ages,
          'Professional':con_partners,
          'Series':con_series,
          'Number of Dances':con_dances,
          'Highest Score':con_highscore,
          'Lowest Score':con_lowscore,
          'Avg Score':con_avscore,
          'PlacingString':con_placings}

# Put that table into a dataframe
df = pd.DataFrame(allinfo)

# Make a copy in case I need to go back and see what it originally was
df_Original = df.copy()

# Drop the preceeding "=" for tied places
df['PlacingString']=(df['PlacingString'].str.lstrip("="))

# Convert the placing column to a more machine-readable format
df['Placing']=(df['PlacingString'].str.split("th").str[0])
df['Placing']=(df['Placing'].str.split("nd").str[0])
df['Placing']=(df['Placing'].str.split("st").str[0])
df['Placing']=(df['Placing'].str.split("rd").str[0])

# Convert to the right datatypes
df = df.astype({'Age':'int32','Series':'int32','Number of Dances':'int32','Highest Score':'int32', \
                 'Lowest Score':'int32','Avg Score':'float','Placing':'int32'})

# Add a "year of competition" column
df['Competing Year']= np.where(df['Series']==1,2004,df['Series']+2002)

# Get rid of the old Placing column
df.drop(['PlacingString'],axis=1,inplace = True)

## Section 2: Going to individuals' wiki pages to pull cross-info from them 

For this, I'm just going to wiki/persons_name, and all the info (as far as I can see) is in the info box down the right hand side. It tends to be there for most of the competitors, so I think there's little enough leakage that I won't go looking any harder for it right now!

In [273]:
df2 = df.copy()

In [274]:
# A test function, to make sure everyone has a wiki page that I can get to
def HasAWikiPage(name):
    # First, get their wiki page up
    spaces = name.replace(" ","_")
    wiki_url = "https://en.wikipedia.org/wiki/" + spaces
    # print(wiki_url)
    page = requests.get(wiki_url)
    
    # If it can't find the page, don't do anything else
    if page.status_code != 200:
        return(0)
    
    return(1)

In [275]:
# A quick test shows that 99% of my contestants have at least a wiki page I can pull up - fab!

df_Test=pd.DataFrame(df['Name'],columns=["Name"])
df_Test['WikiPage']=df_Test['Name'].apply(lambda x:HasAWikiPage(x))
#df_Test.describe()

# Although, hilariously, Alex Jones (that's the only one I've spotted, there may be more) is pulling up
#   the far-right conspiracy theorist of the same name from the US. Not sure what to do about that one!

In [276]:
def GetMarriageStatus(index):
    # First, get their wiki page up
    their_name = df.iloc[index,0]
    # print(their_name + ":\n") 
    wiki_url = "https://en.wikipedia.org/wiki/" + their_name.replace(" ","_")
    page = requests.get(wiki_url)
    
    # If it can't find the page, don't do anything else
    if page.status_code != 200:
        return(0)
    
    # Find the "Spouse(s)" bit of their bio
    soup = BeautifulSoup(page.content,'html.parser')
    
    try:
        infobox = soup.find(class_="infobox biography vcard")
    except AttributeError:
        return(0)
    
    #print(infobox.prettify())
    
    try:
        infolines = infobox.find_all("tr")
    except AttributeError:
        return(0)
    
    # Put the headings in as a list
    stringrows=[]
    for i in range(0,len(infolines)):
        stringrows.append(infolines[i].get_text())
        # Then if we have the "Spouse" row, return it
        if "Spouse(s)" in stringrows[i]:
            spousetext = stringrows[i].lstrip("Spouse(s)")
            return(spousetext)
        
    return(0)

In [277]:
# Put the spouses info in the new table
df2['Spouses String']=(df2.index.map(lambda x:GetMarriageStatus(x)))

## Section 3: Processing the data I already have now, to allow me to answer the question I wanted to

In [286]:
# New dataframe with only the ones with marriage information
df3 = df2[df2['Spouses String']!=0]
df3.head(5)

,Name,Occupation,Age,Professional,Series,Number of Dances,Highest Score,Lowest Score,Avg Score,Placing,Competing Year,Spouses String
1,David Dickinson,TV presenter,63,Camilla Dallerup,1,3,21,16,19.00,7,2004,"Lorne Lesley (b. Irene Spettie,[1] m. 1968)"
2,Verona Joseph,Actress,30,Paul Killick,1,4,27,24,24.75,6,2004,Farren Blackburn (2004—)
7,Natasha Kaplinsky,TV presenter,31,Brendan Cole,1,12,37,26,33.00,1,2004,Justin Bower (m. 2005)
9,Carol Vorderman,TV presenter,43,Paul Killick,2,2,22,20,21.00,9,2004,Christopher Mather(1985–86; divorced)Patrick K...
10,Esther Rantzen,TV presenter,64,Anton du Beke,2,3,24,16,18.66,8,2004,Desmond Wilcox(m. 1977; died 2000)


In [287]:
# A function to return a tuple of the marriage and divorce years.
# Then the odd ones are marriages and the even ones are divorces!

def MarDivList(index,whichone):
    # Spouse String
    ss = df3.iloc[index,11]
    
    # I'm really looking for either 2-digit or 4-digit numerical sequences
    years = []
    i=0
    while i in range(0,len(ss)-1):
        if ss[i:i+2].isnumeric():
            if ss[i:i+4].isnumeric():
                years.append(int(ss[i:i+4]))
                i=i+4
            else:
                years.append(1900+int(ss[i:i+2]))
                i=i+2
        i=i+1
    
    while len(years)<8:
        years.append(0)
    
    return(years[whichone])
        

In [288]:
# Add all the marriage & divorce columns

df3 = df3.reset_index(drop=True)
df3['First Marriage']=(df3.index.map(lambda x:MarDivList(x,0)))
df3['First Divorce']=(df3.index.map(lambda x:MarDivList(x,1)))
df3['Second Marriage']=(df3.index.map(lambda x:MarDivList(x,2)))
df3['Second Divorce']=(df3.index.map(lambda x:MarDivList(x,3)))
df3['Third Marriage']=(df3.index.map(lambda x:MarDivList(x,4)))
df3['Third Divorce']=(df3.index.map(lambda x:MarDivList(x,5)))

df3['Fourth Marriage']=(df3.index.map(lambda x:MarDivList(x,6)))
df3['Fourth Divorce']=(df3.index.map(lambda x:MarDivList(x,7)))

# And then lost the string about the spouses
df3=df3.drop('Spouses String',axis=1)

In [289]:
df3.head(3)

,Name,Occupation,Age,Professional,Series,Number of Dances,Highest Score,Lowest Score,Avg Score,Placing,Competing Year,First Marriage,First Divorce,Second Marriage,Second Divorce,Third Marriage,Third Divorce,Fourth Marriage,Fourth Divorce
0,David Dickinson,TV presenter,63,Camilla Dallerup,1,3,21,16,19.00,7,2004,1968,0,0,0,0,0,0,0
1,Verona Joseph,Actress,30,Paul Killick,1,4,27,24,24.75,6,2004,2004,0,0,0,0,0,0,0
2,Natasha Kaplinsky,TV presenter,31,Brendan Cole,1,12,37,26,33.00,1,2004,2005,0,0,0,0,0,0,0


## Section 4: Dropping the people outside of our time ranges
I don't need the information about people who were already divorced, or weren't yet married at the time of their competition - so get rid!

In [290]:
df4=df3.copy()

In [291]:
df4 = df4.drop(['Occupation','Age','Series','Number of Dances','Placing',\
               'Highest Score','Lowest Score'],axis=1)
df4.head(3)

,Name,Professional,Avg Score,Competing Year,First Marriage,First Divorce,Second Marriage,Second Divorce,Third Marriage,Third Divorce,Fourth Marriage,Fourth Divorce
0,David Dickinson,Camilla Dallerup,19.00,2004,1968,0,0,0,0,0,0,0
1,Verona Joseph,Paul Killick,24.75,2004,2004,0,0,0,0,0,0,0
2,Natasha Kaplinsky,Brendan Cole,33.00,2004,2005,0,0,0,0,0,0,0


In [292]:
def MarriedAtTheTime(start,end,comp):
    if ((comp>=start and comp<=end)or(start != 0 and comp>=start and end==0)):
        return(1)
    else: return(0)
    
def WithinXYears(div,comp,x):
    if (div != 0 and div-comp <=x and comp<=div):
        return(1)
    else: return (0)

In [293]:
df4['Married at the Time']= \
    df4.apply(lambda x: MarriedAtTheTime(x['First Marriage'], x['First Divorce'],x['Competing Year']), axis=1) \
    + df4.apply(lambda x: MarriedAtTheTime(x['Second Marriage'], x['Second Divorce'],x['Competing Year']), axis=1)\
    + df4.apply(lambda x: MarriedAtTheTime(x['Third Marriage'], x['Third Divorce'],x['Competing Year']), axis=1)\
    + df4.apply(lambda x: MarriedAtTheTime(x['Fourth Marriage'], x['Fourth Divorce'],x['Competing Year']), axis=1)

In [294]:
df4['Within 1 Year']= \
    df4.apply(lambda x: WithinXYears(x['First Divorce'], x['Competing Year'],1), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Second Divorce'], x['Competing Year'],1), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Third Divorce'], x['Competing Year'],1), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Fourth Divorce'], x['Competing Year'],1), axis=1) \

df4['Within 2 Years']= \
    df4.apply(lambda x: WithinXYears(x['First Divorce'], x['Competing Year'],2), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Second Divorce'], x['Competing Year'],2), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Third Divorce'], x['Competing Year'],2), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Fourth Divorce'], x['Competing Year'],2), axis=1) \

df4['Within 3 Years']= \
    df4.apply(lambda x: WithinXYears(x['First Divorce'], x['Competing Year'],3), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Second Divorce'], x['Competing Year'],3), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Third Divorce'], x['Competing Year'],3), axis=1) \
    + df4.apply(lambda x: WithinXYears(x['Fourth Divorce'], x['Competing Year'],3), axis=1) \

In [297]:
testingdf = df4[df4['Married at the Time']==1]
testingdf.describe()

,Avg Score,Competing Year,First Marriage,First Divorce,Second Marriage,Second Divorce,Third Marriage,Third Divorce,Fourth Marriage,Fourth Divorce,Married at the Time,Within 1 Year,Within 2 Years,Within 3 Years
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.0,57.000000,57.000000,57.000000
mean,26.246316,2011.175439,1997.964912,807.473684,526.157895,70.035088,70.263158,70.421053,35.245614,35.263158,1.0,0.140351,0.140351,0.140351
std,6.180154,4.480604,14.402337,990.524701,888.277522,370.531759,371.739854,372.576116,266.098551,266.231004,0.0,0.350438,0.350438,0.350438
min,15.000000,2004.000000,1959.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
25%,20.250000,2007.000000,1989.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
50%,25.800000,2011.000000,2001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
75%,30.330000,2015.000000,2008.000000,2000.000000,1976.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
max,36.440000,2019.000000,2019.000000,2017.000000,2019.000000,1996.000000,2008.000000,2014.000000,2009.000000,2010.000000,1.0,1.000000,1.000000,1.000000


In [298]:
df4[df4['Within 1 Year']!=0].head(20)

,Name,Professional,Avg Score,Competing Year,First Marriage,First Divorce,Second Marriage,Second Divorce,Third Marriage,Third Divorce,Fourth Marriage,Fourth Divorce,Married at the Time,Within 1 Year,Within 2 Years,Within 3 Years
22,Letitia Dean,Darren Bennett,29.45,2007,2002,2007,0,0,0,0,0,0,1,1,1,1
39,Jimi Mistry,Flavia Cacace,28.33,2010,2001,2010,2013,0,0,0,0,0,1,1,1,1
41,Patsy Kensit,Robin Windsor,28.00,2010,1988,1991,1992,1996,1997,2000,2009,2010,1,1,1,1
53,Denise van Outen,James Jordan,35.13,2012,2009,2013,0,0,0,0,0,0,1,1,1,1
57,Rachel Riley,Pasha Kovalev,25.33,2013,2012,2013,2019,0,0,0,0,0,1,1,1,1
71,Kirsty Gallacher,Brendan Cole,21.67,2015,2010,2015,0,0,0,0,0,0,1,1,1,1
74,Helen George,Aljaž Skorjanec,33.00,2015,2011,2015,0,0,0,0,0,0,1,1,1,1
79,Louise Redknapp,Kevin Clifton,35.88,2016,1998,2017,0,0,0,0,0,0,1,1,1,1


## Section 6: Getting figures

"In England and Wales in 2017, only 8.4 per 1,000 opposite-sex couples got divorced."*
I'm going to use this figure for now because I can't see any better ones for more recent years.

*https://www.theguardian.com/lifeandstyle/2018/dec/09/in-it-for-the-long-haul-why-divorce-rates-are-falling-fast

In [320]:
# Divorce rate in England and Wales on average
ew_rate = 8.4/1000
ew_percent=str(round(ew_rate*100,2))+"%"

In [321]:
# Divorce rate among married-at-the-time Strictly contestants

scd_rate = df4['Within 1 Year'].sum()/df4['Married at the Time'].sum()
scd_percent=str(round(scd_rate*100,2))+"%"

In [322]:
print("England and Wales divorce percentage in a year: " + ew_percent)
print("Strictly Come Dancing divorce percentage in a year: " + scd_percent)

England and Wales divorce percentage in a year: 0.84%
Strictly Come Dancing divorce percentage in a year: 14.04%


In [324]:
print("Number of times higher Strictly is than normal people: " + str(round(scd_rate/ew_rate,2)))

Number of times higher Strictly is than normal people: 16.71


I want to say I've gone wrong somewhere and it's not NEARLY 17 times higher?? But it feels like it might be...

## Statistical Significant Test

I'm going for a 1-sample test (classic) just to test whether the sample we've picked could possibly have come from a population with a mean of 0.84%, as the ONS published. To be honest, I suspect I haven't got enough datapoints to make them statistically significant.

In [380]:
# Using a 1-sample mean test
# http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS4-OneSampleTtest/SAS4-OneSampleTtest6.html

# Isolate the already married ones
dfX = df4[df4['Married at the Time']==1]

In [377]:
sample_mean = dfX['Within 1 Year'].mean()
sample_std_dev = (dfX['Within 1 Year'].std())
sample_variance = math.sqrt(sample_std_dev)

print("Sample mean: "+str(sample_mean))
print("Sample variance: "+str(math.sqrt(sample_std_dev)))

Sample mean: 0.14035087719298245
Sample variance: 0.5919783121240432


In [378]:
import math

sample_statistic = (sample_mean-ew_rate)/(sample_variance/math.sqrt(57))

"Then reject the null if the sample statistic is more than 1.96 or less than -1.96."

In [379]:
print(sample_statistic)

1.6828442123518463


Our sample statistic is inside the range -- I think I need more data!

## Next Steps:
First, I'd like to make a few graphics and things to show this.

I'd like to look at other celebrities, and see if their rates are higher too (to check that it's the "Strictly Curse", rather than just the "Fame Curse".)

Finally, I think I could read more data in (I've only got 53 people by the end who were married at the time of competing) from other DWTS competitions.